# Solar Tracker
Jupyter notebook for calculating the location of the sun for any date and time, 
given the observers location on the earths surface.

Before I became a software developer, I was a sculptor. I specialised in high-precision sundials, and traded under the name 'Precision Sundials'.

This was in the age before smartphones, and when measuring up for large public pieces, I often spent a long time with calculator, maps and a theodolite that I had purchased in an army surplus store. This meant that if I arrived on site on a sunny day with this kit, I could calculate True north to a fraction of a degree, something that cannot be done with a magnetic compass.

This notebook is a workbook I will use to refer to in creating the Android app that follows.
If smartphones had been available then, and if I had the programming skills that I have now, my life would have been so much easier...

Anyway, to just use this, edit the value of 'now' in the formula below. You will, of course, need to be running this as a Jupyter notebook.

## Setup

We will add the basic information such as date, time and location here, to keep them all together...

Use degrees decimal for latitude and longitude, to prevent confusion over terms 'minutes' and 'seconds', which are also angular measures. In this notbook, they will be used for time only, and angles will be displayed and entered by user in degrees decimal. Calculations will use radians instead.

In [1]:
from datetime import datetime
import datetime as dt
import calendar
import math

now = datetime(2020, 6, 22, 12, 0, 0)
print('Selected Time:', now)

latitude, longitude = 51.9, 7.5
print('Location:', latitude, longitude)


Selected Time: 2020-06-22 12:00:00
Location: 51.9 7.5


## The position of earth relative to the sun

In this section, we take the line from sun to earth on new years day (Gregorian) as the origin. 

We calculate the angular distance that the earth has swept from this origin at the current date and time. 

This will be measured in radians for ease of calculation.

How many seconds in this year?

In [2]:
is_leap = calendar.isleap(now.year)
length_of_year = 366 if is_leap else 365
print('Number of days in this year', length_of_year)

# Mulitply number of days in year by number of seconds in a day
# to get number of seconds in a year.
length_of_year *= 24 * 60 * 60
print('Number of seconds in this year', length_of_year)

Number of days in this year 366
Number of seconds in this year 31622400


How many seconds have passed since noon on January first?

In [3]:
jan1 = datetime(now.year, 1, 1, 12, 0, 0)
seconds_elapsed = now.timestamp() - jan1.timestamp()
print(f'Seconds elapsed {seconds_elapsed}')

Seconds elapsed 14943600.0


Express this in radians, giving the the angle off the sun relative to noon Jan 1

In [4]:
fraction = seconds_elapsed/length_of_year
print(f'As a fraction of a year: {fraction:.4f}')
print(f'In degrees: {fraction * 360:.1f}')

angle = fraction * 2 * math.pi
print(f'In radians: {angle:.4f}')

As a fraction of a year: 0.4726
In degrees: 170.1
In radians: 2.9692


## Longitude, Time Zones

For simplicity, the previous section discounted your location on the surface of the earth, and artificialities introduced with Time Zones (https://en.wikipedia.org/wiki/Time_zone). 

My sister lives in Greenwich. As it happens, Greenwich is at zero degrees longitude. No corrections need be made if you live on the same meridian (line north to south) as Greenwich.

I live in Youghal, Ireland, 7.5 degrees west of Greenwich. Because we live in the same Time Zone, our clocks tell the same time, but solar phenomena such as times of sunset, sunrise and noon (when the sun is due south) are all a half hour later here than in Greenwich.

If she phones me at sunset in Greenwich, the sun is still above the horizon here, and will take another half hour before it sets here. For my sister, the sun is due south at exactly 12 pm. However, it will not be due south of me until 12:30, because the earth has a bit more spinning to do before that can happen. Both our clocks say 12 pm. For her the sun is south. For me, it will not be south for another half hour.

As a Sundial Maker, we use the term 'Local Time' to refer to the time as told by the sun. In Local Time, noon is always 12 pm- 12 pm local time is the time when the sun is south. For me, that happens at 12.30 pm clock time. The difference between clock time and local time is exaclty a half hour, because I live 7.5 degrees west of Greenwich.

The rule for converting from Greenwich to Local time is: Subtract 4 minutes for every degree west of Greenwich to convert from Greenwich to Local time.
-    1/4 degree => 1 minute
-    1 degree   => 4 minutes
-    15 degrees => 1 hour
 
What does this mean for you? It means you must make an adjustmet to the time on your clock to correct it clock time. Look up your longitude on the map, and enter it below in degrees decimal format (maps often don't use degrees decimal, so you may need to convert). Use a negative number if west of Grenwich, positive if to the east.


In [5]:
seconds = longitude * 4 * 60
diff = dt.timedelta(seconds = seconds)
local = now + diff
print('Clock Time', now)
print('Local time',local)

Clock Time 2020-06-22 12:00:00
Local time 2020-06-22 12:30:00


## Apologies...

The sections 'Equation of Time' and 'Declination of the Sun' contain formula with lots of 'magic numbers'. I have included a description of what they are calculating, what they are for, but I have not explained how they have been derived.

I don't want to make promises I don't keep, but I would like to show the derivation of these in a separate notebook, but for now, let's stick to the point.

## The Equation of Time

In this section, we make a small adjustment to the angle we just calculated. 
This corrects a slight discrepancy between clock and solar time. 

It takes the same values at any given date year after year. The maximum correction occurs on 3 November, and amounts to just under sixteen and a half minutes, meaning that solar time is ahead of clock time by about 16 minutes on this date every year. 

The correction varies as the year progresses, and is the same on any given calendar date year after year, give or take a few seconds.

For a good account of the Equation of Time, see https://en.wikipedia.org/wiki/Equation_of_time. In summary, the equation combines two calculations: one is seasonal and purely caused by the geometry of a rotating sphere (the earth), the other is a smaller asymmetrical adjustment due to the fact that the orbit is not perfectly circular.

This section can be ignored if you do not need a high-accuracy calculation. For example, if you are simply measuring the length of a shadow, this correction will have little impact on the calculated length, and can be ignored. 

However, on a sundial, a difference of a quarter hour can be noticeable. For this reason, the Prudent Sundialmaker always includes a graph of the Equation of Time on his sundials. 

The calculation below show the number of minutes solar time is ahead of clock time, if positive. A positive value means that you need to add this number of minutes to clock time to get solar time. A negative value means you must subtract to get solar time.

In [6]:
eq_time = 229.18 * (0.000075 + 0.001868 * math.cos(angle) - 0.032077 * math.sin(angle) - 0.014615 * 
                    math.cos(2 * angle) - 0.040849 * math.sin(2 * angle) )
print(f'Equation of Time, in minutes: {eq_time:.1f}')

Equation of Time, in minutes: -1.7


## Declination of the sun

In this section, we caclulate the declination of the sun. This is the band of latitude over which the sun shines directly at this time of year.

At equinoxes, this is zero, meaning the sun is directly over the equator. At solstices, it reaches it's maximum and minimum values of 23.7 degrees north or south of the equator, depending on which solstice (south in midwinter, north in midsummer).


In [7]:
declination = 0.006918 - 0.399912 * math.cos(angle) + 0.070257 * math.sin(angle) 
- 0.006758 * math.cos(2 * angle) + 0.000907 * math.sin(2 * angle) 
- 0.002697 * math.cos(3 * angle) + 0.00148 * math.sin (3 * angle)

print(f'Declination radians: {declination:.4f}')
print(f'Declination degrees: {declination * 180/math.pi:.1f}')

Declination radians: 0.4130
Declination degrees: 23.7
